In [7]:
from bloqade import squin
from kirin.dialects.ilist import IList
import numpy as np
import bloqade.stim
import bloqade.tsim

In [11]:
# small helper function to take in true/false samples and get back +-1 eigenvalue
def get_eigenvals_from_tf(sample_list):
    result = 1 - 2 * sample_list.astype(int)
    return result

# get X syndromes from measuring the entangled |+> state of auxilary qubits in the z basis.
"""
eigenvalues is a list of +-1's for each qubit, stabilizers is a list of lists of ints, where each one represents
an index where the stabilizer operator is applied (e.g. X_1 X_3 X_5 X_7, X_4 X_5 X_6 X_7, X_2 X_3 X_6 X_7 becomes 
[[0,2,4,6],[3,4,5,6],[1,2,5,6]] (0-indexed)
Output is a list of +-1's, one for each stabilizer.
"""
def get_syndromes(eigenvalues, stabilizers):
    syndrome = []
    for stab in stabilizers:
        eigval = 1
        for idx in stab:
            eigval *= eigenvalues[idx]
        syndrome.append(eigval)
        syndrome = tuple(syndrome)
    return syndrome


""" This function takes 4 arguments:
    1. current_pauli_list, a list of 0's and 1's indicating what the current fixes based on error syndromes are. 
    For example, if we ran the syndrome extraction once, and got a single X gate on qubit 1 then, we run it again and get another X gate, we want to cancel them and apply nothing.
    2. syndrome, list of +-1's
    3. syndrome_table, a dictionary where keys are the error syndromes (tuples of +-1's), and the values are the qubits to apply gates to fix
    4. gate_type, a character determining the gate to apply, either X or Z for the 7 color code.
"""
def update_pauli_corrections_from_syndrome(current_pauli_list, syndrome, syndrome_table):
    index = syndrome_table[syndrome]
    if current_pauli_list[index] == 0:
        current_pauli_list[index] = 1
    else:
        current_pauli_list[index] = 0

In [12]:
# This is the syndrome to correction table for the 7 color code. It is symmtric in X and Z so it works for both.
syndrome_table = {(-1,1,1):0, (1,1,-1):1, (-1,1,-1):2, (1,-1,1):3, (-1,-1,1):4, (1,-1,-1):5, (-1,-1,-1):6}

In [13]:
l = [0,0,0,0,0,0,0]
update_pauli_corrections_from_syndrome(l, (-1,1,1), syndrome_table)
print(l)


[1, 0, 0, 0, 0, 0, 0]
